In [63]:
import yaml
import os

In [64]:
harness_path = '/NS/llm-1/work/afkhan/lm-evaluation-harness/lm_eval/tasks/custom_ppl_task/'
yaml_path = '/NS/llm-1/work/afkhan/Perplexity-vs-Evaluation/Experiments/Perplexity_Runs/EvalHarness_Chat_YAMLs/'
data_path = '/NS/llm-1/work/afkhan/Perplexity-vs-Evaluation/Experiments/Perplexity_Runs/Data/'

In [65]:
data_files = os.listdir(data_path)
# keep those which contain _no_template
data_files = [f for f in data_files if '_no_template' in f]

In [66]:
# task: custom_ppl_task
# output_type: loglikelihood_rolling
# training_split: null
# validation_split: null
# test_split: train
# doc_to_text: ""
# doc_to_target: !function preprocess_wikitext.apply_aya_chat_template
# process_results: !function preprocess_wikitext.process_results
# should_decontaminate: false
# doc_to_decontamination_query: None
# metric_list:
#   - metric: word_perplexity
#   - metric: byte_perplexity
#   - metric: bits_per_byte

# dataset_path: json
# dataset_name: null

# dataset_kwargs:
#   data_files: /NS/llm-1/work/afkhan/Perplexity-vs-Evaluation/Experiments/Perplexity_Runs/Data/hellaswag_okapi_no_template_aya_23_8B.jsonl
#   trust_remote_code: true

In [67]:
def apply_aya_chat_template(doc):
    base_str = doc['text']
    # Example Output - <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Roof shingle removal: A man is sitting on a roof. He<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>
    template_applied_str = '<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>' + base_str + '<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>'
    return template_applied_str

In [68]:
# class CustomDumper(yaml.SafeDumper):
#     def represent_function(self, value):
#         return self.represent_scalar('!function', value)

In [69]:
class FunctionTag(yaml.YAMLObject):
    yaml_tag = '!function'
    
    def __init__(self, function_name):
        self.function_name = function_name

    @classmethod
    def to_yaml(cls, dumper, data):
        # Create a scalar node with the function tag and the function name
        return dumper.represent_scalar(cls.yaml_tag, data.function_name, style=None)

# def make_yaml(task_name, data_files, save_path):
#     task = {
#         'task': task_name,
#         'output_type': 'loglikelihood_rolling',
#         'training_split': None,
#         'validation_split': None,
#         'test_split': 'train',
#         'doc_to_text': FunctionTag("preprocess_wikitext.apply_aya_chat_template"),
#         'process_results': FunctionTag("preprocess_wikitext.process_results"),
#         'should_decontaminate': False,
#         'doc_to_decontamination_query': None,
#         'metric_list': [
#             {'metric': 'word_perplexity'},
#             {'metric': 'byte_perplexity'},
#             {'metric': 'bits_per_byte'}
#         ],
#         'dataset_path': 'json',
#         'dataset_name': None,
#         'dataset_kwargs': {
#             'data_files': data_files,
#             'trust_remote_code': True
#         }
#     }

#     with open(save_path, 'w') as file:
#         yaml.dump(task, file, default_flow_style=False, default_style=None)


In [70]:
def make_yaml(task_name, data_files, save_path):
    task = {
        'task': task_name,
        'output_type': 'loglikelihood_rolling',
        'training_split': None,
        'validation_split': None,
        'test_split': 'train',
        'doc_to_text': "",
        'doc_to_target': FunctionTag("preprocess_wikitext.apply_aya_chat_template"),
        'process_results': FunctionTag("preprocess_wikitext.process_results"),
        'should_decontaminate': False,
        'doc_to_decontamination_query': None,
        'metric_list': [
            {'metric': 'word_perplexity'},
            {'metric': 'byte_perplexity'},
            {'metric': 'bits_per_byte'}
        ],
        'dataset_path': 'json',
        'dataset_name': None,
        'dataset_kwargs': {
            'data_files': data_files,
            'trust_remote_code': True
        }
    }

    with open(save_path, 'w') as file:
        yaml.dump(task, file, default_flow_style=False, default_style=None)

In [71]:
# CustomDumper.add_representer(str, CustomDumper.represent_function)

In [72]:
for d_file in data_files:
    # Remove samples_ prefix
    file_name = d_file.replace('samples_', '')
    # Split on _2024
    file_name = file_name.split('_2024')[0]
    file_name += '_chat_ppl'
    make_yaml(file_name, os.path.join(data_path, d_file), os.path.join(yaml_path, f'{file_name}.yaml'))

In [73]:
# Copy all the yaml files to the harness path
import shutil
for yaml_file in os.listdir(yaml_path):
    shutil.copyfile(os.path.join(yaml_path, yaml_file), os.path.join(harness_path, yaml_file))